In [1]:
# Import delle librerie necessarie
import torch
import numpy as np
import random
import pandas as pd
import wandb
import os

print("Librerie caricate.")

Librerie caricate.


### Impostazioni per la riproducibilità 

In [2]:
def set_seed(seed_value=42):
    """Imposto i seed per la riproducibilità."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        # Imposto anche i seed per la GPU, se disponibile
        torch.cuda.manual_seed_all(seed_value)

# Esegui l'impostazione del seed
set_seed(42) 
print("Random seeds impostati su 42.")

Random seeds impostati su 42.


### Configurazione Iniziale degli Hyperparameters (W&B)

Questa sezione definisce i principali parametri di addestramento (*hyperparameters*) per il *fine-tuning* del modello **ModernBERT**, registrandoli in **Weights & Biases** per tracciabilità e riproducibilità.

| Parametro | Valore | Motivazione della Scelta |
| :--- | :--- | :--- |
| **`learning_rate`** | `5e-5` (0.00005) | È il tasso di apprendimento *default* consigliato da Google/Hugging Face per il *fine-tuning* dei modelli BERT/RoBERTa. È un valore conservativo che assicura che il modello si adatti al nuovo *task* (ACOS) senza "dimenticare" le conoscenze acquisite nel *pre-training*. |
| **`epochs`** | `5` | Un valore tipico e contenuto per il *fine-tuning* di modelli Transformer. Di solito, 3 o 4 *epochs* sono sufficienti per convergere, ma 5 offrono un buon equilibrio tra addestramento e prevenzione dell'overfitting sui dataset di dimensioni limitate come ACOS. |
| **`batch_size`** | `16` | Questa dimensione del *batch* è comune quando si lavora con modelli grandi come RoBERTa-base, bilanciando la stabilità dell'addestramento con le limitazioni della **memoria GPU**. |
| **`model_name`** | `answerdotai/ModernBERT-base` | Scegliamo questo modello specifico Encoder-only come "ModernBERT" per le prestazioni superiori e un pre-training più robusto rispetto al BERTbase originale |
| **`dataset`** | `Laptop-ACOS` | Identifica il sotto-dataset specifico utilizzato per questo esperimento. |
| **`seed`** | `42` | Il **seed di riproducibilità**, fissato a 42 (la convenzione standard ML), garantisce che ogni volta che lo *script* viene eseguito, l'inizializzazione dei pesi e lo *shuffling* dei dati siano identici, garantendo la tracciabilità scientifica dei risultati. |

---

In [3]:
WANDB_ENTITY = "cristinatextmining"

# 1. Definizione degli Hyperparameters
config = {
    "learning_rate": 5e-5,
    "epochs": 5,
    "batch_size": 16,
    "model_name": "answerdotai/ModernBERT-base",
    "dataset": "Laptop-ACOS",
    "seed": 42
}

# 2. Inizializzazione del Run
wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    config=config,
    name=f"run_{config['model_name']}_{config['dataset']}"
)

print(f"W&B inizializzato per il progetto: {wandb.run.project}")

wandb: Currently logged in as: cristinatomaciello2001 (cristinatextmining) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B inizializzato per il progetto: BigData-TextMining-ACOS


### Caricamento dei 3 dataset LAPTOP-ACOS
Questa cella è dedicata al caricamento dei dataset TSV di ACOS (Training, Development e Test) dalla directory locale.

Data la struttura complessa del file di annotazione, che contiene tabulazioni (\t) interne e un numero variabile di quadruple per riga, la funzione load_as_single_string adotta una strategia di caricamento flessibile:

1. **Forzatura Stringa Unica:** Viene utilizzato un separatore inesistente (\x07) per istruire Pandas a caricare l'intera riga TSV come una singola colonna stringa *(full_line_data).*

2. **Robustezza:** Questo approccio previene i comuni ParserError causati da tabulazioni o delimitatori sporchi interni, garantendo che i dati grezzi vengano letti completamente senza perdita.

L'output di questa cella sono i tre DataFrame *(df_train, df_dev, df_test)*, ciascuno pronto per il parsing sequenziale sul contenuto della colonna full_line_data.

In [38]:
import pandas as pd
import os
import sys

# Definisci il percorso base della cartella dei dati
DATA_DIR = 'data/Laptop-ACOS/'

# Definisci i percorsi completi dei file
TRAIN_FILE_PATH = os.path.join(DATA_DIR, 'laptop_train_quad_bert.tsv')
DEV_FILE_PATH = os.path.join(DATA_DIR, 'laptop_dev_quad_bert.tsv')
TEST_FILE_PATH = os.path.join(DATA_DIR, 'laptop_test_quad_bert.tsv')

def load_as_single_string(path):
    """
    Carica l'intero file TSV in un'unica colonna stringa, forzando la lettura riga per riga, 
    per evitare errori di parsing dovuti a delimitatori interni.
    """
    # 1. Utilizza read_csv ma forza l'uso di un separatore inesistente e non usare header
    df = pd.read_csv(
        path, 
        sep='\x07', # Separa per un carattere inesistente (Bell character)
        header=None, 
        on_bad_lines='skip', # Ignora le righe che danno problemi di formattazione
        engine='python'
    )
    
    # Rinomina l'unica colonna che contiene l'intera riga TSV
    df.columns = ['full_line_data']    
    print(f"File {path.split('/')[-1]} caricato come {df.shape[1]} colonna unica.")
    return df

# Esecuzione del caricamento corretto
try:
    df_train = load_as_single_string(TRAIN_FILE_PATH)
    df_dev = load_as_single_string(DEV_FILE_PATH)
    df_test = load_as_single_string(TEST_FILE_PATH)
    
    print("Caricamento Flessibile Completato.")
    
except Exception as e:
    print(f"Errore critico durante il caricamento: {e}")
    raise

df_train.head(5).to_string()

File laptop_train_quad_bert.tsv caricato come 1 colonna unica.
File laptop_dev_quad_bert.tsv caricato come 1 colonna unica.
File laptop_test_quad_bert.tsv caricato come 1 colonna unica.
Caricamento Flessibile Completato.


'                                                                                                                                                                                               full_line_data\n0                                                                                                 ace ##r wants $ 170 to just look at it then add the repair cost on top of that .\\t0,2 SUPPORT#PRICE 1 -1,-1\n1                                                                                                                                      update : i repaired it myself for $ 12 .\\t-1,-1 LAPTOP#GENERAL 1 -1,-1\n2                                                                                                                 i had nothing to lose since it was a paper weight otherwise .\\t-1,-1 LAPTOP#GENERAL 0 -1,-1\n3                                               the shame of it is knowing it took me 15 minutes and $ 12 to fix it and ace ##r wanted to rob me of $ 170 just to look a

### Pipeline di Parsing e Strutturazione dei Dati(Quadruple)
Questa sezione implementa la pipeline di pre-elaborazione finale che trasforma la riga grezza di input in un formato Python strutturato, pronto per la Tokenizzazione del modello ModernBERT.

Il codice esegue un processo a due fasi per ogni riga del dataset:

1. **Separazione (Parsing della Riga Grezza)**: Esegue la divisione della riga unica *(full_line_data)* utilizzando il separatore Tab (\t). Questo isola la recensione pulita *(review_text)* dalla stringa contenente le quadruple codificate (il target).

2. **Decodifica Strutturale (Deep Parsing)**: Applica la funzione *parse_target_quadruples* alla stringa target. Questa funzione scompone la stringa in un dizionario con chiavi chiare **(span_A, span_B, category_aspect, sentiment)**, gestendo gli Indici Span Nullo (-1,-1) e validando la struttura a 4 elementi.

L'output finale è il DataFrame pulito con la colonna parsed_quadruples, che contiene le informazioni di target necessarie per creare le label di Sequence Labeling nella fase successiva.


In [40]:
import pandas as pd
import re
import numpy as np

# --- I. DEFINIZIONE DELLE FUNZIONI DI PARSING ---

def parse_target_quadruples(target_string):
    """
    Decodifica la stringa di una singola quadrupla in un dizionario strutturato, 
    gestendo gli indici nulli (-1,-1).
    """
    if not target_string:
        return {} 

    parts = target_string.split()
    
    # La struttura corretta per la quadrupla è 4 elementi space-separated
    if len(parts) != 4:
        return {}
    
    # Assumiamo l'ordine: Span A, Aspect#Category, Sentiment, Span B
    span_A_str, category_aspect, sentiment_str, span_B_str = parts

    def parse_span(span_str):
        """Converte la stringa degli indici span (es. '1,2') in una tupla di interi."""
        if span_str == '-1,-1':
            return (-1, -1)
        try:
            start, end = map(int, span_str.split(','))
            return (start, end)
        except ValueError:
            return (-1, -1)

    span_A = parse_span(span_A_str)
    span_B = parse_span(span_B_str)
    
    try:
        sentiment = int(sentiment_str)
    except ValueError:
        sentiment = -1 

    return {
        'span_A': span_A,
        'span_B': span_B,
        'category_aspect': category_aspect,
        'sentiment': sentiment
    }


def apply_deep_parsing(raw_quadruples_list):
    """Applica la funzione di deep parsing a tutti gli elementi di una lista di quadruple."""
    parsed_list = []
    for raw_q in raw_quadruples_list:
        parsed_list.append(parse_target_quadruples(raw_q))
    return parsed_list


def parse_full_line(full_line_string):
    """
    Funzione principale: esegue lo split sul Tab ('\\t') e separa il testo dalle quadruple.
    Gestisce anche le quadruple multiple.
    """
    
    parts = full_line_string.split('\t', 1) 
    
    if len(parts) != 2:
        return full_line_string.strip(), []

    review_text = parts[0].strip()
    target_string_raw = parts[1].strip()
    
    target_string_clean = re.sub(r'[\r\n\s]\d+$', '', target_string_raw).strip()
    
    raw_quadruples_list = [q.strip() for q in target_string_clean.split('\t') if q.strip()]

    return review_text, raw_quadruples_list


def apply_full_parsing_pipeline(df):
    """
    Applica l'intera pipeline di parsing, rendendola IDEMPOTENTE:
    esegue il parsing solo se la colonna sorgente 'full_line_data' è presente.
    """
    
    # --- CONTROLLO DI SICUREZZA IDEMPOTENTE ---
    if 'full_line_data' not in df.columns:
        print("AVVISO: Il DataFrame è già stato processato (manca 'full_line_data'). Salto il parsing.")
        return df
    # ------------------------------------------

    df_copy = df.copy() 
    
    # Passaggio 1: Split principale (Crea 'review_text' e 'raw_quadruples_list')
    df_copy[['review_text', 'raw_quadruples_list']] = df_copy['full_line_data'].apply(
        lambda x: pd.Series(parse_full_line(x))
    )
    df_copy = df_copy.drop(columns=['full_line_data'])
    
    # Passaggio 2: Parsing Approfondito (Crea 'parsed_quadruples')
    df_copy['parsed_quadruples'] = df_copy['raw_quadruples_list'].apply(apply_deep_parsing)
    
    return df_copy.drop(columns=['raw_quadruples_list'])


# --- II. ESECUZIONE SEQUENZIALE FINALE ---

print("Avvio della pipeline di parsing sui dataset...")

# Esecuzione del Parsing (Ora è sicuro rieseguire questa cella!)
df_train = apply_full_parsing_pipeline(df_train)
df_dev = apply_full_parsing_pipeline(df_dev)
df_test = apply_full_parsing_pipeline(df_test)

print("Parsing completato su tutti i set (Train, Dev, Test).")

# Verifica finale
print("\nAnteprima del DataFrame di Training con le quadruple decodificate:")
display(df_train[['review_text', 'parsed_quadruples']].head())

Avvio della pipeline di parsing sui dataset...
AVVISO: Il DataFrame è già stato processato (manca 'full_line_data'). Salto il parsing.
AVVISO: Il DataFrame è già stato processato (manca 'full_line_data'). Salto il parsing.
AVVISO: Il DataFrame è già stato processato (manca 'full_line_data'). Salto il parsing.
Parsing completato su tutti i set (Train, Dev, Test).

Anteprima del DataFrame di Training con le quadruple decodificate:


,review_text,parsed_quadruples
0,ace ##r wants $ 170 to just look at it then ad...,"[{'span_A': (0, 2), 'span_B': (-1, -1), 'categ..."
1,update : i repaired it myself for $ 12 .,"[{'span_A': (-1, -1), 'span_B': (-1, -1), 'cat..."
2,i had nothing to lose since it was a paper wei...,"[{'span_A': (-1, -1), 'span_B': (-1, -1), 'cat..."
3,the shame of it is knowing it took me 15 minut...,"[{'span_A': (18, 20), 'span_B': (-1, -1), 'cat..."
4,first one that they shipped was obviously defe...,"[{'span_A': (-1, -1), 'span_B': (7, 8), 'categ..."


### Tokenizzazione e Allineamento delle Label
Questo codice serve per convertire i dati dal formato strutturato di Pandas *(review_text e parsed_quadruples)* nel formato numerico accettato dal modello ModernBERT per l'addestramento.

Il processo si svolge in tre passaggi chiave:

1. **Tokenizzazione:** La recensione pulita *(review_text)* viene convertita in una sequenza di *Input IDs* (numeri) utilizzando il tokenizer di ModernBERT.

2. **Allineamento degli Span:** Utilizziamo le funzioni integrate di Hugging Face per mappare gli Indici Span grezzi ((0, 2), etc.) agli indici dei nuovi subword tokens generati dal tokenizer. Questo passaggio risolve i problemi creati dai ## (tokenizzazione subword).

3. **Sequence Labeling (Codifica BIO):** Sulla base degli indici allineati, creiamo l'array finale di Label Numeriche per ogni token della recensione (es. B-ASPECT, I-OPINION, O).

Questo output (Input IDs e Label Sequence) è la forma finale del dataset, pronto per essere passato al Trainer di ModernBERT per il fine-tuning.